In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

from lion_pytorch import Lion

In [3]:
from vae import VAE
from vae import GroupSoftmax
from trainer import Trainer

In [5]:
# Device config 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [9]:
group_sizes = [3, 6, 6, 12, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
test = VAE(439, 600, 6, 250, group_sizes)

total_params = sum(p.numel() for p in test.parameters())
trainable_params = sum(p.numel() for p in test.parameters() if p.requires_grad)
print(trainable_params)


9666417


In [7]:
# Dataset import 
# Windows path
# data = pd.read_csv('A:/csv_hus/psam_hus_pus_filtered.csv')
# WSL path
data = pd.read_csv('/mnt/a/csv_hde/one_hot_data.csv')


In [8]:
cols = list(data.columns)
cols = [col.split(":")[0] for col in cols]

onehot_counts = {col: sum(data.columns.str.startswith(f"{col}:")) for col in cols}
print(list(onehot_counts.values()))

print(sum(onehot_counts.values()))

[3, 6, 6, 12, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
439


In [27]:
# Data loader 

data_tensor = torch.tensor(data.values, dtype=torch.float32)
data_loader = DataLoader(data_tensor, batch_size=len(data_tensor), shuffle=True)

# Class_sizes 
group_sizes = list(onehot_counts.values())


In [ ]:
model = VAE(439, 550, 6, 250, group_sizes)
optimizer = Lion(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, device)

In [ ]:
# train the model 
model_path = '/mnt/a/models/best_model.pth'
trainer.train(data_loader, 1000, model_path)

Epoch 0, Loss: 0.016557734459638596
Epoch 1, Loss: 0.016559438779950142
Epoch 2, Loss: 0.016560964286327362
Epoch 3, Loss: 0.016564922407269478
Epoch 4, Loss: 0.01657041162252426
Epoch 5, Loss: 0.016579311341047287
Epoch 6, Loss: 0.016587654128670692
Epoch 7, Loss: 0.016600845381617546
Epoch 8, Loss: 0.01660963147878647
Epoch 9, Loss: 0.0166268702596426
Epoch 10, Loss: 0.016645513474941254
Epoch 11, Loss: 0.016662299633026123
Epoch 12, Loss: 0.01668478362262249
Epoch 13, Loss: 0.016709329560399055
Epoch 14, Loss: 0.01673448085784912
Epoch 15, Loss: 0.0167702604085207
Epoch 16, Loss: 0.016796225681900978
Epoch 17, Loss: 0.016829101368784904
Epoch 18, Loss: 0.016870496794581413
Epoch 19, Loss: 0.01690337434411049
Epoch 20, Loss: 0.01693505421280861
Epoch 21, Loss: 0.01696949452161789
Epoch 22, Loss: 0.016999462619423866
Epoch 23, Loss: 0.01702060177922249
Epoch 24, Loss: 0.017049791291356087
Epoch 25, Loss: 0.017086835578083992
Epoch 26, Loss: 0.017128119245171547
Epoch 27, Loss: 0.01715

In [ ]:
torch.save(model.state_dict(), 'model.pth')